# Homework_2: Аналитика текста

## Часть 1: Анализ текста с помощью TF-IDF

### 1. Рассчитайте метрику TF-IDF для любых трех песен на выбранном вами языке

Вес задания: 30%

In [14]:
import glob
import re
from IPython.display import display

import numpy as np
import pandas as pd

import spacy

In [3]:
# Открыть файлы с песнями
file_paths = glob.glob('data/song*.txt')
pattern = r"\[.*\]"

texts = []

for path in file_paths:
    with open(path, 'r', encoding='utf-8') as f:
        text = f.read()
        text = re.sub(pattern, '', text)
        texts.append(text)

In [4]:
import spacy

In [5]:
nlp = spacy.load("ru_core_news_md")

In [6]:
def process_text(text):
    text = text.lower()
    doc = nlp(text)
    lemma_doc = [token.lemma_ for token in doc if
                 not token.is_punct
                 and not token.is_space
                 and not token.is_stop]
    return lemma_doc

texts_lemma = [process_text(t) for t in texts]

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [18]:
texts_joined = [" ".join(text) for text in texts_lemma]

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(texts_joined)
feature_names = vectorizer.get_feature_names_out()
texts_vector = tfidf_matrix.toarray()

for i in range(tfidf_matrix.shape[0]):
    weights = tfidf_matrix[i].toarray().flatten()
    
    df = pd.DataFrame({'word': feature_names,
                       'tfidf': weights})
    
    df = df[df['tfidf'] > 0].sort_values(by='tfidf', ascending=False)
    df = df.reset_index(drop=True)

    print(f"song{i}.txt")
    display(df.head())

,word,tfidf
0,почитай,0.318650
1,старший,0.318650
2,важный,0.136564
3,величина,0.136564
4,мемасиков,0.136564


,word,tfidf
0,видеть,0.221600
1,эмси,0.147733
2,рэпер,0.147733
3,рэп,0.147733
4,искать,0.110800


,word,tfidf
0,этаж,0.281674
1,тело,0.281674
2,гараж,0.211256
3,дешёвый,0.211256
4,микрозелень,0.211256


In [ ]:
# Преобразование в нижний регистр
# Удаление стоп-слов
# Лемматизация/стемминг
# Токенизация

# sklearn.feature_extraction.text
# Расчет матрицы TF-IDF
# Визуализация результатов

### 2. Сравните TF-IDF с другими методами векторизации текста, такими как Count Vectorizer, Word2Vec или Doc2Vec

Вес задания: 30%

### 3. Проведите исследование, используя полученные преобразованные данные

Какие слова/фразы встречаются чаще всего, а какие реже.

Вес задания: 20%

In [ ]:
# Отображение 10 наиболее часто встречающихся слов
# Отображение 10 наиболее часто встречающихся сочетаний слов
# Графическая визуализация (t-sne, WordCloud)


## Часть 2: Классификация тональности с помощью BERT

IMDB Dataset https://disk.yandex.ru/d/dhKpEgM4rQkLiQ;

In [ ]:
# 1. Data preparation:
# Loading IMDB Dataset https://disk.yandex.ru/d/dhKpEgM4rQkLiQ;
# Text Preprocessing;
# Train/Test Split;

### 4. Используйте предварительно обученную модель BERT для классификации тональности отзывов о фильмах

Вес задания: 20%

In [ ]:
# Тонкая настройка BERT: обучите модель на обучающем наборе данных и оцените ее качество на тестовом наборе данных;
# Оценка метрик:
# Точность
# Точность/Полнота
# F1-мера


# **Описание задания**

**2. Сравните TF-IDF с другими методами векторизации текста, такими как Count Vectorizer, Word2Vec или Doc2Vec. (Вес задачи: 30%)**

⚡️Критерии сравнения:
* Вычислительная сложность
* Качество представления
* Интерпретируемость

Результат части 2:
* ✅ Было проведено сравнение трех методов векторизации в соответствии с указанными критериями сравнения;

Снижение оценки:
* Избыточное количество таблиц и графиков без пояснений;
* Реализовано менее трех методов;
* Не все критерии сравнения реализованы;

**3. Проведите исследование, используя полученные преобразованные данные.

Какие слова/фразы встречаются чаще всего, а какие реже (вес задачи: 20%)**

Результаты части 3:
Выполнен статистический анализ текста;

* ✅ Выполнен статистический анализ текста;
* Отображение 10 наиболее часто встречающихся слов;
* Отображение 10 наиболее часто встречающихся сочетаний слов;
* ✅ Графическая визуализация (t-sne, WordCloud);

Выводы:
* Нет выводов относительно сравнения;
* Графики/таблицы не сравнивались;

**4. Используйте предварительно обученную модель BERT для
классификации тональности отзывов о фильмах. (Вес задачи: 20%)**


Execution algorithm::
* ✅ 1.Data preparation:
*   Loading IMDB Dataset https://disk.yandex.ru/d/dhKpEgM4rQkLiQ;
*   Text Preprocessing;
*   Train/Test Split;
* ✅ 2. Работа с BERT:
* Загрузите предварительно обученную модель BERT и соответствующий ей токенизатор с сайта Hugging Face:
* Hugging Face Transformers
* Токенизация
* Создание классификатора
* Подготовка данных: используйте токенизатор BERT для преобразования текстов в формат входных данных модели (входные идентификаторы, маски внимания и т. д.).

* ✅ 3. Обучение модели BERT:
* Тонкая настройка BERT: обучите модель на обучающем наборе данных и оцените ее качество на тестовом наборе данных;
* Оценка метрик:
* Точность
* Точность/Полнота
* F1-мера

Результаты части 4:
* ✅ Точность на тестовом наборе данных не ниже заданного порогового значения (например, 0,9).
* ✅ F1-мера для положительного класса близка к точности, что указывает на баланс между точностью и полнотой.
* ✅ Модель корректно различает явно положительные и явно отрицательные отзывы при ручной проверке примеров.
* ✅ Результаты стабильны при повторном разделении на обучающую и тестовую выборки (значительного снижения показателей не наблюдается).
* ✅ Время вывода на один отзыв достаточно короткое для практического использования (например, доли секунды на типичном GPU/CPU).
* ✅ Алгоритм обучения модели на основе задачи завершен;
* ✅ Все данные должны сопровождаться выводами, основанными на полученных результатах исследования (2-3 предложения);
